In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimCustomers
AS
WITH rem_dup AS
(
SELECT 
  DISTINCT(customer_id),
  customer_email,
  customer_name,
  Customer_Name_Upper
FROM datamodeling.silver.silver_table
)
SELECT * ,
      row_number() OVER (ORDER BY customer_id) as DimCustomerKey
FROM rem_dup

In [0]:
%sql
select * from datamodeling.gold.DimCustomers


In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimProducts
AS
WITH rem_dup AS
(
SELECT 
  DISTINCT(product_id),
  product_name,
  product_category
FROM 
  datamodeling.silver.silver_table
)
SELECT * ,
      row_number() OVER (ORDER BY product_id) as DimProductKey
FROM rem_dup

In [0]:
%sql
select * from datamodeling.gold.dimproducts

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimPayments
WITH rem_dup AS 
(
SELECT 
  DISTINCT(payment_type)
FROM datamodeling.silver.silver_table
) 
SELECT *, row_number() OVER (ORDER BY payment_type) as DimPaymentKey FROM rem_dup

In [0]:
%sql
SELECT * FROM datamodeling.gold.DimPayments

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimRegions
WITH rem_dup AS 
(
SELECT 
  DISTINCT(country)
FROM datamodeling.silver.silver_table
) 
SELECT *, row_number() OVER (ORDER BY country) as DimRegionKey FROM rem_dup

In [0]:
%sql
SELECT * FROM datamodeling.gold.dimregions

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimSales
AS
SELECT 
 row_number() OVER (ORDER BY order_id) as DimSaleKey,
 order_id,
 order_date,
 customer_id,
 customer_name,
 customer_email,
 product_id,
 product_name,
 product_category,
 payment_type,
 country,
 last_updated,
 Customer_Name_Upper,
 processDate
FROM
  datamodeling.silver.silver_table

In [0]:
%sql
SELECT * FROM datamodeling.gold.dimsales

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.FactSales
AS
SELECT 
  S.DimSaleKey,
  C.DimCustomerKey,
  P.DimProductKey,
  R.DimRegionKey,
  PY.DimPaymentKey,
  F.quantity,
  F.unit_price
FROM 
  datamodeling.silver.silver_table F
LEFT JOIN 
  datamodeling.gold.dimcustomers C
  ON F.customer_id = C.customer_id
LEFT JOIN 
  datamodeling.gold.dimproducts P
  ON F.product_id = P.product_id
LEFT JOIN 
  datamodeling.gold.dimregions R
  ON F.country = R.country
LEFT JOIN 
  datamodeling.gold.dimpayments PY
  ON F.payment_type = PY.payment_type
LEFT JOIN 
  datamodeling.gold.dimsales S
  ON F.order_id = S.order_id

In [0]:
%sql
SELECT * FROM datamodeling.gold.FactSales